In [21]:
import tensorflow as tf
import numpy as np
from keras.datasets import mnist

In [54]:
(X_train, y_train), (X_test, y_test) = mnist.load_data(path="mnist.npz")

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

"""One-hot encode y_train and y_test"""
y_train = np.array([np.array([0]*(y)+[1]+[0]*(10-y-1)) for y in y_train])
y_test  = np.array([np.array([0]*(y)+[1]+[0]*(10-y-1)) for y in y_test])

"""Flatten Xs"""
X_train = np.array([X.flatten() for X in X_train])
X_test  = np.array([X.flatten() for X in X_test])


((60000, 28, 28), (60000,))
((10000, 28, 28), (10000,))


In [56]:
input_size = X_train.shape[1]
classes = 10

X = tf.placeholder("float", [None, input_size])
y = tf.placeholder("float", [None, classes]) # 0 through 9

In [83]:
"""Multilayer perceptron"""
# Hooray, generalized!

activation = tf.nn.relu
layers_sizes = [255, 255]
weights = [tf.Variable(tf.random_normal([prev_layer_size, layer_size])) \
           for prev_layer_size, layer_size in \
           zip([input_size]+layers_sizes, layers_sizes+[classes])]

biases = [tf.Variable(tf.random_normal([layer_size])) for layer_size in layers_sizes+[classes]]

def create_layer(prev_layer, weight, bias, activation=activation):
    layer = tf.add(tf.matmul(prev_layer, weight), bias)
    layer = activation(layer)
    return layer

def layers():
    layers = []
    for ind, layer_size in enumerate(layers_sizes):
        try:
            last_layer = layers[-1]
        except IndexError:
            last_layer = X
            print("raising error!")
        layers.append(create_layer(last_layer, weights[ind], biases[ind]))
    layers.append(tf.add(tf.matmul(layers[-1], weights[-1]), biases[-1]))
    print layers
    return layers

In [87]:
BATCH_SIZE = 100
EPOCHS = 15
LEARNING_RATE = 0.001
ACC_EVERY = 1

pred = layers()[-1]
x_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(x_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for epoch in range(EPOCHS):
    for step in range(len(X_train)//BATCH_SIZE):
        start_ind = (step*BATCH_SIZE) % len(X_train)
        end_ind = ((step+1)*BATCH_SIZE) % len(X_train)
        batch_X, batch_y = X_train[start_ind:end_ind], y_train[start_ind:end_ind]
        sess.run(train_step, feed_dict={X:batch_X, y:batch_y})
    if epoch%ACC_EVERY == 0:
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print "Epoch ", epoch
        print "Testing Acc:", accuracy.eval({X: X_test, y: y_test})
        

 raising error!
[<tf.Tensor 'Relu_53:0' shape=(?, 255) dtype=float32>, <tf.Tensor 'Relu_54:0' shape=(?, 255) dtype=float32>, <tf.Tensor 'Add_82:0' shape=(?, 10) dtype=float32>]
Epoch  0
Testing Acc: 0.8589
Epoch  1
Testing Acc: 0.8939
Epoch  2
Testing Acc: 0.9116
Epoch  3
Testing Acc: 0.9201
Epoch  4
Testing Acc: 0.929
Epoch  5
Testing Acc: 0.9305
Epoch  6
Testing Acc: 0.9373
Epoch  7
Testing Acc: 0.9367
Epoch  8
Testing Acc: 0.9387
Epoch  9
Testing Acc: 0.9382
Epoch  10
Testing Acc: 0.9421
Epoch  11
Testing Acc: 0.9441
Epoch  12
Testing Acc: 0.9441
Epoch  13
Testing Acc: 0.9454
Epoch  14
Testing Acc: 0.9475
